In [108]:
#Import Modules
import pandas as pd
import requests
import cbsodata
import datetime

In [109]:
# Get current date information
now = datetime.datetime.now()

# now.year
# now.month
# now.day
# now.hour
# now.minute

In [110]:
# We want all data from now untill max 3 years ago

yearMin = now.year-3
yearMax = now.year

In [111]:
# Dataset 80590ned

# Table definitions

# Geslacht: T001038 = Totaal (man/vrouw)
# Leeftijd: 52052   = 15 tot 75 jaar
# Perioden: YYYY%%MM
# Seizoengecorrigeerd_2 = x1000_Beroepsbevolking_Seizoengecorrigeerd
# Seizoengecorrigeerd_4 = x1000_Werkzame_Beroepsbevolking_Seizoengecorrigeerd
# Seizoengecorrigeerd_6 = x1000_Werkloze_Beroepsbevolking_Seizoengecorrigeerd
# Seizoengecorrigeerd_8 = Werkloosheidspeercentage_Seizoengecorrigeerd_procenten
# Seizoengecorrigeerd_12 = Bruto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten
# Seizoengecorrigeerd_14 = Netto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten


In [112]:
data = pd.DataFrame(cbsodata.get_data(
    '80590ned',
    filters=f"Geslacht eq 'T001038' and Leeftijd eq '52052   ' and substring(Perioden,0,4) ge '{yearMin}'", # These spaces need to be there, they are part of the value..
    select=["Geslacht", "Leeftijd", "Perioden", "Seizoengecorrigeerd_2", "Seizoengecorrigeerd_4", "Seizoengecorrigeerd_6", "Seizoengecorrigeerd_8", "Seizoengecorrigeerd_12", "Seizoengecorrigeerd_14" ]))

In [113]:
# Remove quaterly and yearly data
data = data[data["Perioden"].str.contains("kwartaal")==False]
data = data[data['Perioden'].map(len) > 4 ]

In [114]:
# Rename columns
data = data.rename(columns={
    "Seizoengecorrigeerd_2": "x1000_Beroepsbevolking_Seizoengecorrigeerd",
    "Seizoengecorrigeerd_4" : "x1000_Werkzame_Beroepsbevolking_Seizoengecorrigeerd",
    "Seizoengecorrigeerd_6" : "x1000_Werkloze_Beroepsbevolking_Seizoengecorrigeerd",
    "Seizoengecorrigeerd_8" : "Werkloosheidspeercentage_Seizoengecorrigeerd_procenten",
    "Seizoengecorrigeerd_12" : "Bruto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten", 
    "Seizoengecorrigeerd_14" : "Netto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten"
})

In [115]:
# Export dataFrame to Excel file
data.to_excel("output/E002-UnemploymentRate.xlsx")  